In [1]:
import time
import json
import datasets
from string import Template

In [2]:
model_id = 'reference'
file_path_answer = f"model_answer/{model_id}.jsonl"

In [3]:
user_prompt = "Переведи документ с Английского языка на Русский язык, сохраняя при этом формат и стиль. В своем ответе пиши только перевод документа, не добавляй никаких дополнительных фраз и комментариев. Документ: \n$text"

In [4]:
ds = datasets.load_dataset("datasets/translate_en_ru_zh_alpaca")

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
ds = ds['test']

In [6]:
ds[0]

{'index': 34124,
 'instruction': {'en': 'Generate a blueprint to build a bird feeder.',
  'ru': 'Создайте чертеж, чтобы построить кормушку для птиц.',
  'zh': '生成一个蓝图以建造一个鸟食器。'},
 'input': {'en': '', 'ru': 'nan', 'zh': ''},
 'output': {'en': 'Here is a simple blueprint for building a wooden bird feeder:\n\nMaterials:\n- 1x6 cedar board\n- 1x8 cedar board\n- 1x2 cedar board\n- 1/4 inch dowel\n- Galvanized nails or screws\n- Plexiglass or polycarbonate sheet\n- Waterproof wood glue\n- Sandpaper\n- Saw\n- Hammer or drill\n\nInstructions:\n\n1. Cut the boards: Using the saw, cut the 1x6 cedar board into two 8-inch pieces for the front and back of the bird feeder. Cut the 1x8 cedar board into two 7-inch pieces for the sides. Cut the 1x2 cedar board into two 12-inch pieces for the roof.\n\n2. Assemble the base: Apply waterproof wood glue to the edges of the front and back pieces and attach them to the sides with nails or screws. Make sure the base is square.\n\n3. Cut the plexiglass: Cut two

In [7]:
#file_path = "model_answer/gemma-2-9b-it.jsonl"
#file_path = "question.jsonl"
# with open(file_path, 'r', encoding='utf-8') as file:
#     for line_number, line in enumerate(file, start=1):
#         # Преобразуем строку в JSON-объект
#         json_object = json.loads(line.strip())
#         print(f"Строка {line_number}: {json_object}")
#         break

In [8]:
def save_to_jsonl_file(data_list, file_path):
    """
    Сохраняет список словарей в .jsonl файл.
    
    :param data_list: Список словарей для сохранения
    :param file_path: Путь к выходному .jsonl файлу
    """
    try:
        # Открываем файл в режиме записи
        with open(file_path, 'w', encoding='utf-8') as file:
            for item in data_list:
                # Преобразуем словарь в JSON-строку и записываем в файл
                json_line = json.dumps(item, ensure_ascii=False)
                file.write(json_line + '\n')
        print(f"Данные успешно сохранены в файл: {file_path}")
    except Exception as e:
        print(f"Произошла ошибка при записи файла: {e}")

In [9]:
template = Template(user_prompt)

In [10]:
list_q = []
list_ref = []
for idx in range(len(ds)):
    it = ds[idx]
    content = it['output']['en']
    content_ru = it['output']['ru']
    content = template.substitute(text=content)
    dict_q = {'question_id': idx, 'cluster':1, 'turns': [{"content": content}], 'category': "translate_alpaca_small"}
    list_q.append(dict_q)
    dict_ref = {'question_id': idx, 'answer_id': idx, 'model_id': model_id, 
                'choices': [{'index': 0, 'turns': [{'content': content_ru, 'token_len': 0}]}], 
                "tstamp": time.time()}
    list_ref.append(dict_ref)
    #break

In [11]:
dict_ref

{'question_id': 0,
 'answer_id': 0,
 'model_id': 'reference',
 'choices': [{'index': 0,
   'turns': [{'content': 'Вот простой чертеж изготовления деревянной кормушки для птиц: Материалы: - кедровая доска 1x6 - кедровая доска 1x8 - кедровая доска 1x2 - дюбель 1/4 дюйма - оцинкованные гвозди или шурупы - лист плексигласа или поликарбоната - водостойкий клей для дерева - наждачная бумага - пила - Молотком или дрелью Инструкции: 1. Разрежьте доски: Используя пилу, разрежьте кедровую доску 1x6 на две 8-дюймовые части для передней и задней части кормушки. Разрежьте кедровую доску 1x8 на две 7-дюймовые части для сторон. Разрежьте кедровую доску 1x2 на две 12-дюймовые части для крыши. 2. Соберите основание: нанесите водостойкий столярный клей на края передней и задней частей и прикрепите их к бокам гвоздями или шурупами. Убедитесь, что основание квадратное. 3. Вырежьте плексиглас: Вырежьте два куска плексигласа или листа поликарбоната, чтобы они соответствовали отверстию в основании, оставив н

In [12]:
save_to_jsonl_file(list_q, "question.jsonl")

Данные успешно сохранены в файл: question.jsonl


In [13]:
save_to_jsonl_file(list_ref, file_path_answer)

Данные успешно сохранены в файл: model_answer/reference.jsonl
